In [28]:
import pandas as pd
from math import log2
import re

## Начальные данные

In [47]:
code = 'БАВАВАВВББВВБВВААВВБАВБААВВАВВАББВВВАБВВБВВАААВБВАВБААВВВАББАБВВВВВВБВААВВБААВАВАВВВВААБВВВААВВВББВА'
code_plusone = 'БАВАВАВВББВВБВВААВВБАВБААВВАВВАББВВВАБВВБВВАААВБВАВБААВВВАББАБВВВВВВБВААВВБААВАВАВВВВААБВВВААВВВББВАБ'
combinations = ['АА', 'АБ', 'АВ', 'БА', 'ББ', 'БВ', 'ВА', 'ВБ', 'ВВ']

## Количество символов

In [101]:
print('La - кол-во символов =', len(code))
la = len(code)

La - кол-во символов = 100


## Параметры алфавита

In [11]:
alphabet = list(set(list(code)))
print('Алфавит =', alphabet)
print('ma - объем алфавита =', len(alphabet))

Алфавит = ['Б', 'А', 'В']
ma - объем алфавита = 3


## Вероятности p(ai)

In [147]:
pa_df = pd.DataFrame({
    'Символ (ai)': alphabet
})
for index, row in pa_df.iterrows():
    letter = row['Символ (ai)']
    pa_df.loc[index, 'Кол-во символов (nai)'] = code.count(letter)
    pa_df.loc[index, 'Вероятность (p(ai))'] = (code.count(letter))/(len(code))
    pa_df.loc[index, 'Кол-во инфы на символ (J(ai))'] = round(-log2((code.count(letter))/(len(code))), 2)

jc = 0
for nai, jai in zip(pa_df['Кол-во символов (nai)'].tolist(), pa_df['Кол-во инфы на символ (J(ai))'].tolist()):
    jc += nai * jai
print('J сообщения =', jc)
print('J сообщения для сравнения =', jc/100)

pa_df.loc['Сумма']= pa_df.sum()
pa_df

J сообщения = 148.6
J сообщения для сравнения = 1.486


,Символ (ai),Кол-во символов (nai),Вероятность (p(ai)),Кол-во инфы на символ (J(ai))
0,Б,20.0,0.2,2.32
1,А,30.0,0.3,1.74
2,В,50.0,0.5,1.00
Сумма,БАВ,100.0,1.0,5.06


## Определение типа источника

In [91]:
aiaj_df = pd.DataFrame({
    'aiaj': combinations,
})
for index, row in aiaj_df.iterrows():
    combination = row['aiaj']
    aiaj_df.loc[index, 'n(aiaj)'] = len(re.findall(f'(?={combination})', code_plusone))
    aiaj_df.loc[index, 'p(aiaj)'] = (len(re.findall(f'(?={combination})', code_plusone)))/len(code)
    aiaj_df.loc[index, 'p(ai) * p(aj)'] = pa_df[pa_df['Символ (ai)']==combination[0]]['Вероятность (p(ai))'].tolist()[0] * pa_df[pa_df['Символ (ai)']==combination[1]]['Вероятность (p(ai))'].tolist()[0]
    aiaj_df.loc[index, 'p(aj|ai)'] = ((len(re.findall(f'(?={combination[::-1]})', code_plusone)))/len(code))/(pa_df[pa_df['Символ (ai)']==combination[1]]['Вероятность (p(ai))'].tolist()[0])
k = 0
for index, row in aiaj_df.iterrows():
    if round(row['p(aiaj)'], 2) == round(row['p(ai) * p(aj)'], 2): 
        k+=1
if k == len(aiaj_df): istochnik = 'Независимый источник'
else: istochnik = 'Марковский источник'

print(istochnik)

aiaj_df.loc['Сумма']= aiaj_df.sum()
aiaj_df

Независимый источник


,aiaj,n(aiaj),p(aiaj),p(ai) * p(aj),p(aj|ai)
0,АА,9.0,0.09,0.09,0.3
1,АБ,6.0,0.06,0.06,0.3
2,АВ,15.0,0.15,0.15,0.3
3,БА,6.0,0.06,0.06,0.2
4,ББ,4.0,0.04,0.04,0.2
5,БВ,10.0,0.10,0.10,0.2
6,ВА,15.0,0.15,0.15,0.5
7,ВБ,10.0,0.10,0.10,0.5
8,ВВ,25.0,0.25,0.25,0.5
Сумма,АААБАВБАБББВВАВБВВ,100.0,1.00,1.00,3.0


## Энтропия

In [149]:
if istochnik == 'Марковский источник':
    pass
    ha = 0
    for i in range(0, len(alphabet)):
        pai = pa_df[pa_df['Символ (ai)']==alphabet[i]]['Вероятность (p(ai))'].tolist()[0]
        logsum = 0
        for j in range(0, len(alphabet)):
            pajai = aiaj_df[aiaj_df['aiaj']==(alphabet[j]+alphabet[i])]['p(aj|ai)'].tolist()[0]
            logsum += pajai * log2(pajai)
            print(pajai, alphabet[j]+alphabet[i])
        ha += logsum * pai
    # print(-ha)
else:
    ha = 0
    for i in range(0, len(alphabet)):
        ha += pa_df[pa_df['Символ (ai)']==alphabet[i]]['Вероятность (p(ai))'].tolist()[0] * log2(pa_df[pa_df['Символ (ai)']==alphabet[i]]['Вероятность (p(ai))'].tolist()[0])
    ha = ha
    print('Энтропия H(A)', ha)
print('Сравить с Jc =', jc/100)

Энтропия H(A) -1.4854752972273344
Сравить с Jc = 1.486


## Избыточность

In [112]:
r = (log2(len(alphabet))-ha)/(log2(len(alphabet)))
print('Избыточность r =', round(r, 2))

Избыточность r = 1.94


## Минимальное кол-во двоичных символов, необходимых для кодирования

In [114]:
lkmin = - la * ha
print('Lkmin =', lkmin)

Lkmin = 148.54752972273343


## Равномерный код

In [135]:
ravnomer_df = pd.concat([pa_df['Символ (ai)'], pa_df['Кол-во символов (nai)']], axis=1)
ravnomer_df = ravnomer_df.drop (index = 'Сумма' )
ravnomer_df['Кодовая комбинация'] = ['00', '01', '11']
ravnomer_df['n(0)'] = [2*20, 1*30, 0]
ravnomer_df['n(1)'] = [0, 1*30, 2*50]
print('p(0)', ravnomer_df['n(0)'].sum()/200)
print('p(1)', ravnomer_df['n(1)'].sum()/200)
ravnomer_df.loc['Сумма']= ['-', ravnomer_df.sum()[1], '-', ravnomer_df.sum()[3], ravnomer_df.sum()[4]]
print('delta p', (130-70)/200)
print('r равномер', round((200-lkmin)/200, 2))
ravnomer_df

p(0) 0.35
p(1) 0.65
delta p 0.3
r равномер 0.26


,Символ (ai),Кол-во символов (nai),Кодовая комбинация,n(0),n(1)
0,Б,20.0,00,40,0
1,А,30.0,01,30,30
2,В,50.0,11,0,100
Сумма,-,100.0,-,70,130


## Эффективный код Шеннона - Фано

In [146]:
shennon_df = pd.concat([pa_df['Символ (ai)'], pa_df['Кол-во символов (nai)'], pa_df['Вероятность (p(ai))']], axis=1)
shennon_df = shennon_df.drop (index = 'Сумма')
shennon_df = shennon_df.sort_values(by='Вероятность (p(ai))', ascending=False)
shennon_df['Кодовая комбинация'] = ['0', '10', '11']
shennon_df['n(0)'] = [1*50, 1*30, 0]
shennon_df['n(1)'] = [0, 1*30, 2*20]
print('p(0)', round(shennon_df['n(0)'].sum()/150, 2))
print('p(1)', round(shennon_df['n(1)'].sum()/150, 2))
shennon_df.loc['Сумма']= ['-', shennon_df.sum()[1], shennon_df.sum()[2], '-', shennon_df.sum()[4], shennon_df.sum()[5]]
print('delta p', round((80-70)/150, 2))
print('r шф', round((150-lkmin)/150, 2))
shennon_df

p(0) 0.53
p(1) 0.47
delta p 0.07
r шф 0.01


,Символ (ai),Кол-во символов (nai),Вероятность (p(ai)),Кодовая комбинация,n(0),n(1)
2,В,50.0,0.5,0,50,0
1,А,30.0,0.3,10,30,30
0,Б,20.0,0.2,11,0,40
Сумма,-,100.0,1.0,-,80,70
